(c) 2016 - present. Enplus Advisors, Inc.

# Programming with Data: Foundations of Pandas & Python

# Split, Apply, Combine

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 2)

In [2]:
dat = pd.read_csv('starmine.csv', parse_dates=['date'])
# dat = dat.set_index(['date', 'symbol'], verify_integrity=True).sort_index()

sectors = 'Durbl Enrgy HiTec'.split(' ')
dates = ['1995-01-31', '1995-02-28']
cols = ['date', 'symbol', 'sector', 'smi', 'ret_0_1_m', 'cap_usd']
dat = dat[cols].query('sector in @sectors and date in @dates').reset_index(drop=True)

## Split, Apply, Combine

Hadley Wickham  
**The split-apply-combine strategy for data analysis.**  
_Journal of Statistical Software_, vol. 40, no. 1, pp. 1–29, 2011  

In [3]:
grp = dat.groupby('sector')

## Split

Conceptually, allows iteration over the split `DataFrame`

In [4]:
for sector_name, sector_df in grp:
    print(f'Sector Name: {sector_name}')
    break # stop the iteration

Sector Name: Durbl


In [5]:
sector_df.head()

,date,symbol,sector,smi,ret_0_1_m,cap_usd
4,1995-01-31,3FDMLQ,Durbl,31.0,0.15,5.01e+08
28,1995-01-31,AICOQ,Durbl,68.0,-0.04,1.68e+08
29,1995-01-31,AIHI,Durbl,45.0,0.11,3.03e+08
48,1995-01-31,APN,Durbl,NaN,0.10,1.24e+08
52,1995-01-31,ARV,Durbl,18.0,-0.02,5.11e+08


## Apply

Default applies to all non-numeric columns.

In [6]:
grp.mean()

,smi,ret_0_1_m,cap_usd
sector,,,
Durbl,47.60,0.01,2.06e+09
Enrgy,31.30,0.05,3.87e+09
HiTec,61.95,0.06,1.48e+09


# Apply

Apply to a single column.

In [7]:
# Returns a Series
grp['smi'].mean()

sector
Durbl    47.60
Enrgy    31.30
HiTec    61.95
Name: smi, dtype: float64

In [8]:
# Returns a DataFrame
grp[['smi']].mean()

,smi
sector,
Durbl,47.60
Enrgy,31.30
HiTec,61.95


# Apply

Apply the same function to multiple *selected* columns.

In [9]:
grp[['smi', 'cap_usd']].mean()

,smi,cap_usd
sector,,
Durbl,47.60,2.06e+09
Enrgy,31.30,3.87e+09
HiTec,61.95,1.48e+09


# Apply

Apply different functions to a single column and give the
result `DataFrame` custom names.

Use the `aggregate` or `agg` method.

# Apply

Apply different functions to a single column. Results have the same
names as the functions.

In [10]:
grp['smi'].agg([np.mean, np.sum, 'std'])

,mean,sum,std
sector,,,
Durbl,47.60,4903.0,27.11
Enrgy,31.30,3287.0,25.96
HiTec,61.95,31161.0,28.49


# Apply

Apply different functions to different columns.

Again, use the `aggregate` or `agg` method.

In [11]:
grp.agg({
    'smi': lambda x: x.mean(),
    'cap_usd': np.std
}).rename(columns={
    'smi': 'mean_smi',
    'cap_usd': 'mean_cap_usd'
})

,mean_smi,mean_cap_usd
sector,,
Durbl,47.60,5.05e+09
Enrgy,31.30,1.06e+10
HiTec,61.95,4.48e+09


## Grouping by Multiple Variables

Same idea as before, except our results now have a MultiIndex.

In [12]:
grp2 = dat.groupby(['sector', 'date'])
grp2[['cap_usd', 'smi']].mean()

cap_usd    smi
sector date                       
Durbl  1995-01-31  1.97e+09  50.58
       1995-02-28  2.15e+09  44.44
Enrgy  1995-01-31  3.79e+09  28.49
       1995-02-28  3.94e+09  34.17
HiTec  1995-01-31  1.43e+09  61.98
       1995-02-28  1.53e+09  61.92

# Flexible Apply

Use `apply` to operate on each grouped subset of the `DataFrame`

In [13]:
grp3 = dat.groupby('sector')
grp3.apply(lambda df: pd.Series(df.shape, index=['nrow', 'ncol']))

,nrow,ncol
sector,,
Durbl,131,6
Enrgy,164,6
HiTec,672,6
